In [1]:
import sys
from pathlib import Path
import os
import pandas as pd

sys.path.append(os.getcwd())

# notebooks/03_modeling.ipynb → spaceship-titanic
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)
print("SRC EXISTS =", (PROJECT_ROOT / "src").exists())


from src.preprocessing import preprocess_data


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier


PROJECT_ROOT = c:\KC\spaceship-titanic
SRC EXISTS = True


In [2]:

import pandas as pd

train_df = pd.read_csv("../data/raw/train.csv")
test_df = pd.read_csv("../data/raw/test.csv")

from src.preprocessing import preprocess_data
X = preprocess_data(train_df.drop("Transported", axis=1))
y = train_df["Transported"]

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)




c:\KC\spaceship-titanic\src\preprocessing.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["HomePlanet"].fillna("Unknown", inplace=True)
c:\KC\spaceship-titanic\src\preprocessing.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [3]:
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,CabinNum,Side,TotalSpend,GroupSize
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,B,0.0,P,0.0,1
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,F,0.0,S,736.0,1
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,A,0.0,S,10383.0,2
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,A,0.0,S,5176.0,2
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,F,1.0,S,1091.0,1


In [4]:
print(train_df.shape, test_df.shape)


(8693, 14) (4277, 13)


PREPROCESSING PIPELINE

In [5]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "bool"]).columns

preprocessor = ColumnTransformer([
    ("num", SimpleImputer(strategy="median"), num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]), cat_cols)
])


BASELINE MODEL (Logistic Regression)

In [6]:
from sklearn.linear_model import LogisticRegression

baseline_model = LogisticRegression(max_iter=1000)

baseline_pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", baseline_model)
])

baseline_pipe.fit(X_train, y_train)

baseline_preds = baseline_pipe.predict(X_val)
print("Baseline Accuracy:", accuracy_score(y_val, baseline_preds))


Baseline Accuracy: 0.7855089131684876


c:\KC\spaceship-titanic\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

rf_pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("model", rf_model)
])

rf_pipe.fit(X_train, y_train)

rf_preds = rf_pipe.predict(X_val)
print("Random Forest Accuracy:", accuracy_score(y_val, rf_preds))


Random Forest Accuracy: 0.7998849913743531


In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_model = XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

xgb_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", xgb_model)
])

xgb_pipeline.fit(X_train, y_train)

xgb_preds = xgb_pipeline.predict(X_val)

xgb_acc = accuracy_score(y_val, xgb_preds)
print("XGBoost Accuracy:", xgb_acc)


XGBoost Accuracy: 0.8177113283496262


In [9]:
import os

os.makedirs("models", exist_ok=True)


import joblib

joblib.dump(xgb_pipeline, "models/xgb_model.pkl")


['models/xgb_model.pkl']

In [10]:
xgb_pipeline.fit(X_train, y_train)
xgb_preds = xgb_pipeline.predict(X_val)

print(
    "XGBoost Accuracy (with GroupSize):",
    accuracy_score(y_val, xgb_preds)
)

XGBoost Accuracy (with GroupSize): 0.8177113283496262


In [11]:
X_test = preprocess_data(test_df)

c:\KC\spaceship-titanic\src\preprocessing.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["HomePlanet"].fillna("Unknown", inplace=True)
c:\KC\spaceship-titanic\src\preprocessing.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [12]:
test_preds = xgb_pipeline.predict(X_test)

In [13]:
test_preds = test_preds.astype(bool)

In [14]:
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Transported": test_preds
})

In [15]:
submission.to_csv("submission.csv", index=False)